In [1]:
# import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
# import our first source file

# source file 1, the whole neighborhood file
Beverly_Grove_Total_df = pd.read_csv('Neighborhood_Files/Beverly_Grove_Finished.csv', index_col='PARCEL')
Beverly_Grove_Total_df.head()

,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,MAILSTATE,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,CA,...,2.0,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,CA,...,1.0,0,0,0,0,0,0,YES,0,0
5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,CA,...,1.0,0,0,0,0,0,0,YES,0,0
5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,CA,...,5.0,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,CA,...,1.0,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


In [3]:
# import our second source file

# source file 2, the sales for that neighborhood
Beverly_Grove_Sales_df = pd.read_csv('Sales_Files/Beverly_Grove_Sales.csv', index_col='PARCEL')
Beverly_Grove_Sales_df.head()

,Owner Name2,Owner1 First Name,Owner1 Last Name,Site Address,Site Address House Number,Site Address Unit Number,Site Address Street Name,Site Address City,Site Address State,Site Address Zip,...,Building Area,Sales Price,Tax Delinquent,Tax Exemption Code,Tract,Use Code,Use Code Description,Census Tract,Year Built,Zoning
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Astrid Meghrigian,Astrid,Meghrigian,NaN,NaN,NaN,NaN,Los Angeles,CA,90036,...,6100.0,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2
5086013015,6000 Sv Holdings Llc,NaN,6000 Sv Holdings Llc,NaN,NaN,NaN,NaN,Los Angeles,CA,90036,...,5000.0,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2
5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,NaN,3Rd St,Los Angeles,CA,90036,...,9487.0,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1
5511028030,Hart Third Street Llc,NaN,Hart Third Street Llc,8038 W 3Rd St,8038.0,NaN,3Rd St,Los Angeles,CA,90048,...,2205.0,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2
5511029032,Triple Heights Llc,NaN,Triple Heights Llc,8124 W 3Rd St,8124.0,NaN,3Rd St,Los Angeles,CA,90048,...,6400.0,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2


In [4]:
# create a new column which will later be transformed into our output column after we've merged the two df's

Beverly_Grove_Sales_df['Did it sell?'] = 'yes'

# check the new column
Beverly_Grove_Sales_df.head()

,Owner Name2,Owner1 First Name,Owner1 Last Name,Site Address,Site Address House Number,Site Address Unit Number,Site Address Street Name,Site Address City,Site Address State,Site Address Zip,...,Sales Price,Tax Delinquent,Tax Exemption Code,Tract,Use Code,Use Code Description,Census Tract,Year Built,Zoning,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Astrid Meghrigian,Astrid,Meghrigian,NaN,NaN,NaN,NaN,Los Angeles,CA,90036,...,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2,yes
5086013015,6000 Sv Holdings Llc,NaN,6000 Sv Holdings Llc,NaN,NaN,NaN,NaN,Los Angeles,CA,90036,...,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2,yes
5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,NaN,3Rd St,Los Angeles,CA,90036,...,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1,yes
5511028030,Hart Third Street Llc,NaN,Hart Third Street Llc,8038 W 3Rd St,8038.0,NaN,3Rd St,Los Angeles,CA,90048,...,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2,yes
5511029032,Triple Heights Llc,NaN,Triple Heights Llc,8124 W 3Rd St,8124.0,NaN,3Rd St,Los Angeles,CA,90048,...,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2,yes


In [5]:
# drop the useless columns, i.e only keep 'PARCEL' and 'Did it sell?'

Beverly_Grove_Sales_df = Beverly_Grove_Sales_df[['Did it sell?']]

In [6]:
# check to see if it worked.

Beverly_Grove_Sales_df.head()

,Did it sell?
PARCEL,
5089002004,yes
5086013015,yes
5512016010,yes
5511028030,yes
5511029032,yes


# Merge our dataframes

In [7]:
Beverly_Grove_Total_df = Beverly_Grove_Total_df.merge(Beverly_Grove_Sales_df, how='left', left_index=True, right_index=True)
Beverly_Grove_Total_df.head()

,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,MAILSTATE,...,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,CA,...,0,1,0,0,YES,CENTRAL,CENTRAL,0,0,NaN
5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,CA,...,0,0,0,0,0,0,YES,0,0,NaN
5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,CA,...,0,0,0,0,0,0,YES,0,0,NaN
5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,CA,...,0,1,0,0,YES,0,CENTRAL,0,0,NaN
5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,CA,...,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0,NaN


In [8]:
# Fill in the NaN's in the 'Did it sell' column
Beverly_Grove_Total_df['Did it sell?'] = Beverly_Grove_Total_df['Did it sell?'].fillna('no')

In [9]:
# check the value counts
Beverly_Grove_Total_df['Did it sell?'].value_counts()

no     6228
yes     502
Name: Did it sell?, dtype: int64

# Drop the columns we don't need

In [10]:
# check columns
Beverly_Grove_Total_df.columns

Index(['Composite Name', 'Owned by Trust?', 'Owned by Business?',
       'SITEADDRESS', 'SITECITY', 'SITEZIP', 'MAILADDRESS',
       'MAIL DIFFERENT FROM SITE?', 'MAILCITY', 'MAILSTATE',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'DEEDTYPE', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'WALLHGT', 'VIEW',
       'Did it sell?'],
      dtype='object')

In [11]:
# drop 'Composite Name', 'SITEADDRESS', 'SITECITY', 'SITEZIP', 'MAILADDRESS','WALLHGT'
Beverly_Grove_Total_df.drop(['Composite Name', 'SITEADDRESS', 'SITECITY', 'SITEZIP',
                             'MAILADDRESS', 'WALLHGT', 'MAILCITY', 'MAILSTATE'],
                              axis=1, inplace=True)

Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,No,No,No,No,FIRST AMERICAN TITLE,1157385,925908,231477,0,0,...,2.0,0,1,0,0,YES,CENTRAL,CENTRAL,0,no
5085033008,No,No,Yes,No,0,57047,40994,16053,0,0,...,1.0,0,0,0,0,0,0,YES,0,no
5085032022,No,No,No,No,ATTORNEY ONLY,651485,521193,130292,0,HOMEOWNERS,...,1.0,0,0,0,0,0,0,YES,0,no
5510005006,Yes,No,Yes,No,0,85697,74539,11158,0,0,...,5.0,0,1,0,0,YES,0,CENTRAL,0,no
5089024002,No,No,No,No,0,62487,42013,20474,0,HOMEOWNERS,...,1.0,1,0,0,YES,YES,CENTRAL,CENTRAL,0,no


# Encode Column Values

In [12]:
# Determine the number of unique values in each column.

Beverly_Grove_Total_df.apply(lambda x: len(x.unique()))

Owned by Trust?                 2
Owned by Business?              2
MAIL DIFFERENT FROM SITE?       2
MAIL OUTSIDE CA?                2
TITLECO1                      119
ASSDTOTAL                    6137
ASSDLAND                     5124
ASSDSTCT                     5603
ASSDOTHR                        5
EXEMPTCD                        2
EXMPTAMT                       14
PCNTIMPD                       94
TAXAMT                       6671
DOCDATE_YEAR                   58
DEEDTYPE                       23
MULTIPARCEL                     3
PRICE                        1172
LENDER1                       504
LOANAMOUT1                   1394
LOANTYPE1                      11
YEARBLT                        98
EFFYRBLT                       99
LOTSQFT                      1757
TOTALSF                      2186
GRGTYPE                         7
BLDGMAT                         9
TOTUNITS                       34
QUALCLAS                        6
BLDGCOND                        6
NOSTORY       

In [13]:
# Convert binary value counts to 0's and 1's, since most of them are 'Yes' or 'No' right now.
# Step 1: 'Owned by Trust?'

Beverly_Grove_Total_df['Owned by Trust?'].replace({"Yes": 1, "No": 0}, inplace=True)

In [14]:
# Step 2: 'Owned by Business?'
Beverly_Grove_Total_df['Owned by Business?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 3: 'MAIL DIFFERENT FROM SITE?'
Beverly_Grove_Total_df['MAIL DIFFERENT FROM SITE?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 4: 'MAIL OUTSIDE CA'
Beverly_Grove_Total_df['MAIL OUTSIDE CA?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 5: 'EXEMPTCD'
Beverly_Grove_Total_df['EXEMPTCD'].replace({"HOMEOWNERS": 1}, inplace=True)

# Step 6: 'Did it sell?'
Beverly_Grove_Total_df['Did it sell?'].replace({"yes": 1, "no": 0}, inplace=True)

In [15]:
Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,FIRST AMERICAN TITLE,1157385,925908,231477,0,0,...,2.0,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,1.0,0,0,0,0,0,0,YES,0,0
5085032022,0,0,0,0,ATTORNEY ONLY,651485,521193,130292,0,1,...,1.0,0,0,0,0,0,0,YES,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,5.0,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,1.0,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


In [16]:
Beverly_Grove_Total_df['Owned by Trust?'].value_counts()

0    5033
1    1697
Name: Owned by Trust?, dtype: int64

In [17]:
Beverly_Grove_Total_df['Owned by Trust?'].dtypes

dtype('int64')

In [18]:
Beverly_Grove_Total_df['FAMILYRM'].value_counts()

0    6443
1     284
2       3
Name: FAMILYRM, dtype: int64

# Now encode the categorical data

In [19]:
Beverly_Grove_Total_df.columns

Index(['Owned by Trust?', 'Owned by Business?', 'MAIL DIFFERENT FROM SITE?',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'DEEDTYPE', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW',
       'Did it sell?'],
      dtype='object')

In [20]:
# Those columns will be the following: 
# TITLECO1
# DEEDTYPE
# MULTIPARCEL
# LENDER1
# LOANTYPE1
# GRGTYPE
# BLDGMAT
# QUALCLAS
# BLDGCOND
# POOL
# PATIO
# FIREPLCE
# AIRMTHOD
# HEATMTHD
# VIEW

In [21]:
Encoded_List = [
    'TITLECO1',
    'MULTIPARCEL',
    'LENDER1',
    'LOANTYPE1',
    'GRGTYPE',
    'BLDGMAT',
    'QUALCLAS',
    'BLDGCOND',
    'POOL',
    'PATIO',
    'FIREPLCE',
    'AIRMTHOD',
    'HEATMTHD',
    'VIEW',
    'ROOFMAT',
    'FOUNDATN',
]

In [22]:
for col in Beverly_Grove_Total_df:
    if col in Encoded_List:
        Beverly_Grove_Total_df[col] = Beverly_Grove_Total_df[col].transform(lambda x: x != '0')
        Beverly_Grove_Total_df[col] = Beverly_Grove_Total_df[col].astype(int)


In [23]:
Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,2.0,0,1,0,0,1,1,1,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,1.0,0,0,0,0,0,0,1,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,1.0,0,0,0,0,0,0,1,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,5.0,0,1,0,0,1,0,1,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,1.0,1,0,0,1,1,1,1,0,0


## Now encode 'Deedtype' but to into individual values for 9 largest, plus 'Other' and '0' values. 


In [24]:
Deedtype_Series = Beverly_Grove_Total_df.groupby('DEEDTYPE').count()['ASSDTOTAL'].tail(-10).index
Deedtype_Series

Index(['GRANT DEED', 'IN LIEU OF FORC', 'INTERSPOUSAL', 'JOINT TENANT',
       'NAME CHANGE', 'PARTNERSHIP', 'PERSONAL REP', 'QUIT CLAIM', 'RE-RECORD',
       'TAX DEED', 'TRUST TRANSFER', 'TRUSTEES', 'WARRANTY'],
      dtype='object', name='DEEDTYPE')

In [25]:
Beverly_Grove_Total_df = pd.get_dummies(Beverly_Grove_Total_df, columns=['DEEDTYPE'], prefix="", prefix_sep="")

In [26]:
Beverly_Grove_Total_df.sample(5)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,JOINT TENANT,NAME CHANGE,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
4335012160,0,0,0,0,1,496000,268000,228000,0,0,...,0,0,0,0,0,0,0,0,0,0
5512008031,0,0,0,0,1,1294417,1035534,258883,0,0,...,0,0,0,0,0,0,0,0,0,0
5085017006,0,0,0,0,0,188723,111522,77201,0,0,...,0,0,0,0,0,0,0,0,0,0
5089011039,1,0,1,0,1,422085,118670,303415,0,1,...,0,0,0,0,0,0,0,0,0,0
5512009010,0,0,0,0,1,944324,755460,188864,0,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
Beverly_Grove_Total_df['QUIT CLAIM'].value_counts()

0    5753
1     977
Name: QUIT CLAIM, dtype: int64

In [28]:
Beverly_Grove_Total_df['Did it sell?'].value_counts()

0    6228
1     502
Name: Did it sell?, dtype: int64

In [29]:
# drop any NaN's
Beverly_Grove_Total_df.dropna()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,JOINT TENANT,NAME CHANGE,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335003015,0,1,1,0,0,1177690,1177690,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4335010018,0,1,1,0,1,1346195,1346195,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5510018012,1,0,0,0,0,762455,509154,253301,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
Beverly_Grove_Total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6730 entries, 5085010022 to 5514001029
Data columns (total 65 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Owned by Trust?            6730 non-null   int64  
 1   Owned by Business?         6730 non-null   int64  
 2   MAIL DIFFERENT FROM SITE?  6730 non-null   int64  
 3   MAIL OUTSIDE CA?           6730 non-null   int64  
 4   TITLECO1                   6730 non-null   int64  
 5   ASSDTOTAL                  6730 non-null   int64  
 6   ASSDLAND                   6730 non-null   int64  
 7   ASSDSTCT                   6730 non-null   int64  
 8   ASSDOTHR                   6730 non-null   int64  
 9   EXEMPTCD                   6730 non-null   object 
 10  EXMPTAMT                   6730 non-null   int64  
 11  PCNTIMPD                   6730 non-null   int64  
 12  TAXAMT                     6730 non-null   float64
 13  DOCDATE_YEAR               6730 n

## Create our X and y values

In [31]:
y = Beverly_Grove_Total_df['Did it sell?']
X = Beverly_Grove_Total_df.drop(['Did it sell?'], 1)

## Split our data into a training and testing dataset

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [33]:
X_train

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,JOINT TENANT,NAME CHANGE,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5511012037,1,0,0,0,0,291024,64179,226845,0,1,...,0,0,0,0,0,0,0,1,0,0
5089011151,0,0,0,0,1,600000,387500,212500,0,0,...,0,0,0,0,0,0,0,0,0,0
5510026067,0,0,1,0,1,313000,60800,252200,0,1,...,0,0,0,0,0,0,0,0,0,0
4334023013,1,0,0,0,0,98117,53408,44709,0,1,...,0,0,0,0,0,0,0,0,0,0
4333024045,0,0,1,1,1,487015,323086,163929,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333012060,0,0,0,0,0,1026627,662341,364286,0,0,...,0,0,0,0,0,0,0,0,0,0
4335002038,0,0,1,0,0,758037,430948,327089,0,0,...,0,0,0,0,1,0,0,0,0,0
5511020012,1,0,0,0,0,316230,199031,117199,0,1,...,0,0,0,0,0,0,0,0,0,0


## Scale our data

In [34]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# check out scaled data
X_train_scaled[:2]

array([[ 1.7281701 , -0.19506457, -0.61835346, -0.16891024, -1.40751506,
        -0.69580008, -0.96238589,  0.02485225, -0.01530829,  1.03735258,
         0.09585413,  2.26915999, -0.70925971,  0.16411276, -0.08112689,
        -0.58320408, -0.83934005, -0.58357339, -0.81353386,  0.36243674,
         0.31158033, -0.30587438, -0.06406915, -1.21910086,  0.70731695,
        -0.23950964,  1.5546902 ,  1.55692527,  0.32685151,  0.71426434,
        -1.23225934, -0.77771639, -0.3186284 , -0.20947947, -1.16787668,
        -0.44588379, -0.46333693, -1.24097142,  1.08720499,  0.05813537,
        -0.15064872, -0.37732229, -0.14290046, -0.0199106 , -0.01407753,
        -0.04673621, -0.04881923, -0.05813537, -0.03449986,  0.        ,
        -0.03984491, -1.23788661, -0.01407753, -0.13626116, -0.02816343,
        -0.0199106 , -0.01407753, -0.0199106 , -0.41319009, -0.03984491,
        -0.01407753,  3.66333182, -0.03984491, -0.01407753],
       [-0.57864674, -0.19506457, -0.61835346, -0.16891024,  0.

In [36]:
X_train.shape

(5047, 64)

In [37]:
# check to see that the mean is approximately 0
print(np.mean(X_train_scaled[:,0]))

-2.886095915015267e-17


In [38]:
# check to see that the standard deviation is approximately 1
print(np.std(X_train_scaled[:,0]))

1.0


## Create a learning model

In [39]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5047, 64)
(1683, 64)
(5047,)
(1683,)


In [40]:
X_train_scaled

array([[ 1.7281701 , -0.19506457, -0.61835346, ...,  3.66333182,
        -0.03984491, -0.01407753],
       [-0.57864674, -0.19506457, -0.61835346, ..., -0.27297555,
        -0.03984491, -0.01407753],
       [-0.57864674, -0.19506457,  1.61719804, ..., -0.27297555,
        -0.03984491, -0.01407753],
       ...,
       [ 1.7281701 , -0.19506457, -0.61835346, ..., -0.27297555,
        -0.03984491, -0.01407753],
       [-0.57864674, -0.19506457, -0.61835346, ..., -0.27297555,
        -0.03984491, -0.01407753],
       [-0.57864674, -0.19506457, -0.61835346, ..., -0.27297555,
        -0.03984491, -0.01407753]])

In [41]:
np.isnan(X_train_scaled)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [42]:
y_train

PARCEL
5511012037    0
5089011151    0
5510026067    0
4334023013    0
4333024045    0
             ..
4333012060    0
4335002038    0
5511020012    0
4332018052    0
5510010017    0
Name: Did it sell?, Length: 5047, dtype: int64

In [43]:
pd.isna(X_train_scaled[1])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [50]:
X_train_scaled.dtype

dtype('float64')

In [51]:
print(np.isnan(X_train_scaled))

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [62]:
X_train_scaled.max()

nan

In [64]:
y_train.min()

0

In [69]:
type(X_train_scaled)

numpy.ndarray

In [72]:
X_train_scaled.sort(axis=1, kind=None, order=None)

X_train_scaled

array([[-1.40751506, -1.24097142, -1.23788661, ...,  1.7281701 ,
         2.26915999,  3.66333182],
       [-1.41379335, -1.40004189, -1.24097142, ...,  1.22920514,
         2.13296072,  4.9041349 ],
       [-1.74521321, -1.68706925, -1.41379335, ...,  1.22920514,
         1.61719804,  2.44301827],
       ...,
       [-1.40751506, -1.23788661, -0.83934005, ...,  1.7281701 ,
         1.86495835, 17.20123115],
       [-1.41379335, -1.40751506, -1.40004189, ...,  1.08720499,
         2.38506551,  2.42019358],
       [-1.40751506, -1.23788661, -0.9639924 , ...,  4.14380083,
         4.22840219,  4.25440587]])

In [56]:
from sklearn import tree

# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()

# Fitting the model.
model = model.fit(X_train_scaled, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').